In [190]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
#from data.dataset import DATASET_AUG as aug
from data.dataset import DATASET as dt

In [160]:
aug = aug[aug.columns.drop(["latitude", "longitude"])]

In [161]:
risks = list(dt.filter(like='risk').columns)
aug = aug.append(dt[risks])

In [191]:
#dt = aug.copy()
# dt["risk0"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
1206    NaN
1207    NaN
1208    NaN
1209    0.0
1210    NaN
Name: risk0, Length: 1210, dtype: float64

In [192]:
dt.head()

,city,latitude,longitude,population,country_code,c40,risk0,risk1,risk2,risk3,...,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP),country,population_1k_density
0,Aalborg,57.0337,9.9166,122219.0,DNK,False,NaN,NaN,NaN,NaN,...,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark,1289.757080
1,Aarhus,56.1572,10.2107,237551.0,DNK,False,NaN,NaN,NaN,NaN,...,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark,2450.983398
2,Copenhagen,55.6786,12.5635,1085000.0,DNK,False,NaN,2.0,NaN,2.0,...,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark,6691.528320
3,Esbjerg,55.4670,8.4500,72205.0,DNK,False,NaN,NaN,NaN,NaN,...,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark,854.210083
4,Frederikshavn,57.4337,10.5333,24103.0,DNK,False,NaN,2.0,NaN,NaN,...,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark,551.431763


The dataset includes different risks that need a prediction. Every risk is considered as a different target of labels, namely a response variable.

The aim is to build a model able to predict each risk in the most accurate way possible. However, the learning process is different for each of them, meaning that the minimum set of variables that best explain the largest amount of variance in the dataset is unique for every risk. As a consequence, the following pipeline will be executed as much time as the number of risks in order to return as more precise predictions as possible. 

# Dataset splitting

The first step consists in splitting the dataset into training and test sets. The first will be used during the feature selection part, which is implemented using a boosted logistic regression model. This is a supervised learning approach, thus labels are needed for the regression to be carried out. In this dataset risks are assigned to only some of the cities, therefore it's wise to select as training set all the entries containing values for the given risk. All the rest will be referred to as test set, used for the classification task, since those cities will be the ones needing a prediction.

In [193]:
import random

def data_splitting(dt,risk):
    # Select the columns containing labelled risk remove labels from the dataset to define training set
    train = dt[dt[risk].notnull()]
    y_train = train[risk] # define response variable
    # Remove labels from the dataset to define training set
    train = train[dt.columns.difference(dt.filter(like = 'risk').columns,sort=False)]
    # Remove categorical columns since they are only descriptive
    num_cols = train._get_numeric_data().columns
    to_drop = list(set(train.columns) - set(num_cols))
    to_drop.append("c40") #comment if aug
    train = train[train.columns.drop(to_drop)]
    # Define test set
    test = dt[~dt.index.isin(train.index)]
    test = test[test.columns.drop(to_drop)]
    test = test[test.columns.difference(test.filter(like = 'risk').columns,sort=False)]
    y_test = [random.randrange(4) for x in range(len(test))]
    return train, test, y_train, y_test

# Feature selection

When there is a highly non-linear and complex relationship between the predictors and the labels decision trees are preferable. The dataset has many different predictors and we don't know whether this relationship is linear or not.

The most robust approach among the ensemble method is `Boosting`. It allows to aggregate many decision trees, differently from `Random Forest`, and grow them sequentially, instead of using boostrap sampling like in `Bagging`. 

The procedure consists in fitting small trees to the residuals in order to slowly improve the prediction error. Generally, model that learn slowly tend to perform better. A pitfall of Boosting, however, is that it relies very much on its tuning parameters. Hence, it's important to undergo `Cross Validation` in order to select the combination returning the highest accuracy, for every target. 
For this purpose we decided to use 10-fold cross validation in such a way to speed up the tuning process, which is already slow given the amount of parameters that need to be optimized.

In [89]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
model = xgb.XGBRegressor()

XgBoost has as default objective function `reg:squarederror`, which corresponds to a linear regression with mean-squared error as loss function.

In [196]:
def boosting_reg(train, y_train, risk, best_parameters):
    
    '''Cross Validation'''
    
    kfold = KFold(n_splits=10)
    reg_cv = GridSearchCV(model, cv = kfold,
                          param_grid = {"colsample_bytree":[0.1,0.5,1.0],"min_child_weight":[1.0,1.2],
                            'max_depth': [7,9], 'n_estimators': [500], "alpha": [10,12,15], "subsample": [0.5],
                                    "objective": ["reg:squarederror"]})
                            #"objective": ["multi:softmax", "multi:softprob", "rank:map"], "n_classes": 4'''    
    reg_cv.fit(train,y_train)
    best_parameters[risk] = reg_cv.best_params_
    
    '''Training'''
    
    gbm = xgb.XGBRegressor(**best_parameters[risk])
    gbm.fit(train,y_train)
    
    '''Feature selection
    im=pd.DataFrame({'importance':gbm.feature_importances_,'var':X.columns})
    im=im.sort_values(by='importance',ascending=False)
    fig,ax = plt.subplots(figsize=(8,8))
    plot_importance(xgb,max_num_features=15,ax=ax,importance_type='gain')
    plt.show()'''
    
    # accuracy_scores[risk]=[gbm.score(train,y_train),0]   
    sorted_idx = np.argsort(gbm.feature_importances_)[::-1]
    best_features = list()
    for index in sorted_idx:
        if gbm.feature_importances_[index] > 0:
            best_features.append(train.columns[index]) 
    return gbm, best_features[:15], best_parameters#, accuracy_scores

# Classification

In [91]:
from sklearn.metrics import accuracy_score

In [188]:
def boosting_clas(gbm, test, y_test, risk, accuracy_scores):
    
    predictions = gbm.predict(test, iteration_range = (0, gbm.best_iteration)).argmax(axis=0)
    accuracy_scores[risk] = [gbm.score(test,y_test),0] #old
    accuracy_scores[risk][1] = accuracy_score(y_test, predictions) #new
    
    return predictions, accuracy_scores

# Generation of a reduced dataset filled with predictions

In [197]:
#WIP

risks = ["risk3"]#list(dt.filter(like='risk').columns)
best_parameters = dict()
accuracy_scores = dict()
filled_dataset = dt.copy()

for risk in risks:
    train, test, y_train, y_test = data_splitting(dt,risk)
    gbm, features, best_parameters = boosting_reg(train, y_train, risk, best_parameters)
    fname = risk+"_"+"boost_model.json"
    gbm.save_model(fname)
    # gbm.load_model(fname)

    predictions, accuracy_scores = boosting_clas(gbm, test, y_test, risk, accuracy_scores)
    test_index = dt.index.isin(test.index)
    filled_dataset.loc[test_index, risk] = predictions.round()
    
    get_back = ["city", "country", risk]
    to_drop = set(dt.columns) - set(features) - set(get_back)
    filled_red_dataset = filled_dataset[dt.columns.drop(to_drop)]
    
    filled_red_dataset.to_csv(risk+"_"+'filled_red_dataset.csv',index=False)

TypeError: Singleton array 298 cannot be considered a valid collection.

In [182]:
best_parameters["risk0"]

{'alpha': 15,
 'colsample_bytree': 0.1,
 'max_depth': 7,
 'min_child_weight': 1.0,
 'n_estimators': 500,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [187]:
accuracy_scores

{'risk0': [-0.45550314401437486, 0], 'risk1': [-0.004324828429177252, 0]}